In [1]:
import zipfile
import os

In [8]:
!wget --no-check-certificate \
    "https://github.com/m-sumaim/corn_seed_dataset/archive/refs/heads/main.zip" \
    -O "/tmp/seed_class.zip"


zip_ref = zipfile.ZipFile('/tmp/seed_class.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-08-24 06:04:56--  https://github.com/m-sumaim/corn_seed_dataset/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/m-sumaim/corn_seed_dataset/zip/refs/heads/main [following]
--2023-08-24 06:04:57--  https://codeload.github.com/m-sumaim/corn_seed_dataset/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/seed_class.zip’

/tmp/seed_class.zip     [        <=>         ] 491.66M  26.2MB/s    in 19s     

2023-08-24 06:05:16 (25.9 MB/s) - ‘/tmp/seed_class.zip’ saved [515546839]



In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
#from PIL import Image as image
import numpy as np

# Set the directories for the training and testing data
train_dir = '/tmp/corn_seed_dataset-main/train'
test_dir = '/tmp/corn_seed_dataset-main/validation'

# Create an ImageDataGenerator for the training data with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Create an ImageDataGenerator for the testing data without data augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a train generator that generates batches of augmented images from the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Create a test generator that generates batches of images from the testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Build the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

# Save the trained model
model.save('corn_seed_classifier.h5')

# Load the trained model
model = tf.keras.models.load_model('corn_seed_classifier.h5')

def predict_image(image_path):
    # Load the image and resize it to the required size
    img = image.load_img(image_path, target_size=(150, 150))

    # Convert the image to a numpy array and normalize it
    x = image.img_to_array(img)
    x = x / 255.0

    # Add a new axis to create a batch of size 1
    x = np.expand_dims(x, axis=0)

    # Make a prediction on the image
    predictions = model.predict(x)

    # Get the index of the class with the highest probability
    class_index = np.argmax(predictions[0])

    # Get the name of the class from the class index
    class_names = ['Broken', 'Discolored', 'Pure', 'Silkcut']
    predicted_class_name = class_names[class_index]

    # Get the probability of the predicted class
    class_probability = predictions[0][class_index]

    return predicted_class_name, class_probability

# Make a prediction on the image
image_path = '/home/test_image.png'
predicted_class_name, class_probability = predict_image(image_path)

# Print the predicted class and its probability
print(f'The predicted class is: {predicted_class_name}')
print(f'The probability of the predicted class is: {class_probability:.2f}')


Found 14322 images belonging to 4 classes.
Found 3474 images belonging to 4 classes.
Epoch 1/10
448/448 [==============================] - 1021s 2s/step - loss: 1.2532 - accuracy: 0.4120 - val_loss: 1.2282 - val_accuracy: 0.4067
Epoch 2/10
448/448 [==============================] - 1015s 2s/step - loss: 1.1886 - accuracy: 0.4654 - val_loss: 1.1472 - val_accuracy: 0.4790
Epoch 3/10
448/448 [==============================] - 976s 2s/step - loss: 1.1358 - accuracy: 0.4979 - val_loss: 1.1193 - val_accuracy: 0.4954
Epoch 4/10
448/448 [==============================] - 939s 2s/step - loss: 1.1155 - accuracy: 0.5166 - val_loss: 1.0534 - val_accuracy: 0.5328
Epoch 5/10
448/448 [==============================] - 967s 2s/step - loss: 1.0914 - accuracy: 0.5244 - val_loss: 1.0506 - val_accuracy: 0.5348
Epoch 6/10
448/448 [==============================] - 939s 2s/step - loss: 1.0775 - accuracy: 0.5340 - val_loss: 1.1342 - val_accuracy: 0.4859
Epoch 7/10
448/448 [==============================] - 9

In [12]:
image_path = '/home/broken_test.png'
predicted_class_name, class_probability = predict_image(image_path)

# Print the predicted class and its probability
print(f'The predicted class is: {predicted_class_name}')
print(f'The probability of the predicted class is: {class_probability:.2f}')


1/1 [==============================] - 0s 60ms/step
The predicted class is: Discolored
The probability of the predicted class is: 0.61


In [14]:
from google.colab import files

files.download("corn_seed_classifier.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install gradio

In [22]:
import gradio as gr
import tensorflow as tf

model = tf.keras.models.load_model('/content/corn_seed_classifier.h5')
labels = ['Pure', 'Broken', 'Silkcut', 'Discolored']
no_classes = len(labels)

def classify_img(inp):
    inp = inp.reshape((-1, 224, 224, 3))
    inp = tf.keras.applications.inception_v3.preprocess_input(inp)
    prediction = model.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(no_classes)}

image = gr.inputs.Image(shape=(224, 224))
label = gr.outputs.Label(num_top_classes=4)

gr.Interface(fn=classify_img, inputs=image, outputs=label, interpretation="default").launch(share=True)


<ipython-input-22-35861a9b1f88>:14: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image = gr.inputs.Image(shape=(224, 224))
<ipython-input-22-35861a9b1f88>:14: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image = gr.inputs.Image(shape=(224, 224))
<ipython-input-22-35861a9b1f88>:15: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  label = gr.outputs.Label(num_top_classes=4)
<ipython-input-22-35861a9b1f88>:15: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  label = gr.outputs.Label(num_top_classes=4)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://16dcb5ea49b23b6608.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
